## Importing the required Libraries

In [1]:
# Import
# ======

# essential libraries
import math
import random
import datetime
from datetime import timedelta

# storing and anaysis
import numpy as np
import pandas as pd

# visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import calmap
import folium
import datetime as dt
from datetime import date
import requests
import sys
from itertools import chain
import pycountry
import pycountry_convert as pc

import plotly.io as pio
pio.templates.default = "plotly_dark"
%matplotlib inline

# color pallette
cnf, dth, rec, act, grn = '#393e46', '#ff2e63', '#21bf73', '#fe9801','#2ca02c' 
blu,bro = '#1f77b4','#8c564b'
# converter
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()   

# hide warnings
import warnings
warnings.filterwarnings('ignore')

## Importing the Data

### Data set
#### clean_data
##### Province,Lat,Long,Date,Confirmed,Deaths,Recovered

In [2]:
clean_data=pd.read_csv('covid_19_clean_complete.csv')

### Cleaning Data - Data Processing

In [3]:
display(clean_data.shape)
display(clean_data.info())
display(clean_data.head())
display(clean_data.describe())

(24104, 8)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24104 entries, 0 to 24103
Data columns (total 8 columns):
Province/State    7360 non-null object
Country/Region    24104 non-null object
Lat               24104 non-null float64
Long              24104 non-null float64
Date              24104 non-null object
Confirmed         24104 non-null int64
Deaths            24104 non-null int64
Recovered         24104 non-null int64
dtypes: float64(2), int64(3), object(3)
memory usage: 1.5+ MB


None

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.0000,65.0000,1/22/20,0,0,0
1,NaN,Albania,41.1533,20.1683,1/22/20,0,0,0
2,NaN,Algeria,28.0339,1.6596,1/22/20,0,0,0
3,NaN,Andorra,42.5063,1.5218,1/22/20,0,0,0
4,NaN,Angola,-11.2027,17.8739,1/22/20,0,0,0


,Lat,Long,Confirmed,Deaths,Recovered
count,24104.000000,24104.000000,24104.000000,24104.000000,24104.000000
mean,21.433571,22.597991,2064.253941,121.304389,504.504647
std,24.740933,70.570960,20166.993881,1308.820006,4525.386285
min,-51.796300,-135.000000,-1.000000,-1.000000,0.000000
25%,7.000000,-19.020800,0.000000,0.000000,0.000000
50%,23.659750,20.921188,4.000000,0.000000,0.000000
75%,41.204400,81.000000,155.000000,2.000000,12.000000
max,71.706900,178.065000,839675.000000,46583.000000,99400.000000


In [4]:
clean_data.rename(columns={'Date': 'date', 
                     'Province/State':'state',
                     'Country/Region':'country',
                     'Lat':'lat','Long':'long',
                     'Confirmed': 'confirmed',
                     'Deaths':'deaths',
                     'Recovered':'recovered'
                    }, inplace=True)
# cases 
cases = ['confirmed', 'deaths', 'recovered', 'active']
# Active Case = confirmed - deaths - recovered
clean_data['active'] = clean_data['confirmed'] - clean_data['deaths'] - clean_data['recovered']

In [5]:
clean_data

,state,country,lat,long,date,confirmed,deaths,recovered,active
0,NaN,Afghanistan,33.000000,65.000000,1/22/20,0,0,0,0
1,NaN,Albania,41.153300,20.168300,1/22/20,0,0,0,0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0,0,0,0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0,0,0,0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0,0,0,0
...,...,...,...,...,...,...,...,...,...
24099,Saint Pierre and Miquelon,France,46.885200,-56.315900,4/22/20,1,0,0,1
24100,NaN,South Sudan,6.877000,31.307000,4/22/20,4,0,0,4
24101,NaN,Western Sahara,24.215500,-12.885800,4/22/20,6,0,0,6
24102,NaN,Sao Tome and Principe,0.186360,6.613081,4/22/20,4,0,0,4


#### Date Formatting

In [6]:
clean_data['date'] = pd.to_datetime(clean_data['date'])
today = date.today()
yesterday = today-datetime.timedelta(days=1)
tomorrow = today+datetime.timedelta(days=1)
max_date = clean_data['date'].max()
print("Today's Date is ",today)
print("yesterday's Date is ",yesterday)
print("Tomorrow's Date is ",tomorrow)
print("latest record date",max_date)

Today's Date is  2020-04-24
yesterday's Date is  2020-04-23
Tomorrow's Date is  2020-04-25
latest record date 2020-04-22 00:00:00


#### Checking whether data is updated

In [7]:
# Check if the data is updated
print("External Data")
print(f"Earliest Entry: {clean_data['date'].min()}")
print(f"Last Entry:     {clean_data['date'].max()}")
print(f"Total Days:     {clean_data['date'].max() - clean_data['date'].min()}")

External Data
Earliest Entry: 2020-01-22 00:00:00
Last Entry:     2020-04-22 00:00:00
Total Days:     91 days 00:00:00


#### Cheking for missing values and taking care of them

In [8]:
#Missing Values
clean_data.isnull().sum().to_frame('nulls')

,nulls
state,16744
country,0
lat,0
long,0
date,0
confirmed,0
deaths,0
recovered,0
active,0


In [9]:
clean_data[['state']] = clean_data[['state']].fillna('')
clean_data[cases] = clean_data[cases].fillna(0)

In [10]:
clean_data.isnull().sum().to_frame('nulls')

,nulls
state,0
country,0
lat,0
long,0
date,0
confirmed,0
deaths,0
recovered,0
active,0


#### clean_data - Data Types

In [11]:
clean_data.dtypes

state                object
country              object
lat                 float64
long                float64
date         datetime64[ns]
confirmed             int64
deaths                int64
recovered             int64
active                int64
dtype: object

#### Filtering problemating Data

In [12]:
# Filter out problematic data points (The Diamond Princess, cruise ships were associated with Canada)
removed_state = "Diamond Princess"
removed_country = "Diamond Princess"

clean_data = clean_data[~clean_data["state"].replace(np.nan, "nan").str.match(removed_state)]
clean_data = clean_data[~clean_data["country"].replace(np.nan, "nan").str.match(removed_country)]

In [13]:
# dealing with negative values in active cases (Min in active is -6), this shows that any of the confirmed or deaths or 
# recovered are having negative values 
num1 = clean_data['active']._get_numeric_data()
num1[num1 < 0] = 0

In [14]:
display(clean_data.shape)
display(clean_data.info())
display(clean_data.head())
display(clean_data.describe())

(23920, 9)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23920 entries, 0 to 24103
Data columns (total 9 columns):
state        23920 non-null object
country      23920 non-null object
lat          23920 non-null float64
long         23920 non-null float64
date         23920 non-null datetime64[ns]
confirmed    23920 non-null int64
deaths       23920 non-null int64
recovered    23920 non-null int64
active       23920 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(4), object(2)
memory usage: 1.8+ MB


None

,state,country,lat,long,date,confirmed,deaths,recovered,active
0,,Afghanistan,33.0000,65.0000,2020-01-22,0,0,0,0
1,,Albania,41.1533,20.1683,2020-01-22,0,0,0,0
2,,Algeria,28.0339,1.6596,2020-01-22,0,0,0,0
3,,Andorra,42.5063,1.5218,2020-01-22,0,0,0,0
4,,Angola,-11.2027,17.8739,2020-01-22,0,0,0,0


,lat,long,confirmed,deaths,recovered,active
count,23920.000000,23920.000000,23920.000000,23920.000000,23920.000000,23920.000000
mean,21.598445,22.771822,2078.138796,122.215008,507.430268,1448.495778
std,24.764115,70.813933,20243.769190,1313.803098,4542.590314,16473.372728
min,-51.796300,-135.000000,-1.000000,-1.000000,0.000000,0.000000
25%,7.844875,-20.026050,0.000000,0.000000,0.000000,0.000000
50%,23.692500,20.972650,4.000000,0.000000,0.000000,2.000000
75%,41.227200,81.812500,151.000000,2.000000,12.000000,75.000000
max,71.706900,178.065000,839675.000000,46583.000000,99400.000000,715726.000000


clean_data.head()

#### Creating a Master Copy of the Data

##### Data Set Name - data
###### Columns - province,lat,long,date,confirmed,deaths,recovered.active

In [15]:
data = clean_data.copy()

In [16]:
data.columns

Index(['state', 'country', 'lat', 'long', 'date', 'confirmed', 'deaths',
       'recovered', 'active'],
      dtype='object')

In [17]:
data['mortality'] = data['deaths'] / data['confirmed']

In [18]:
data.sample(10)

,state,country,lat,long,date,confirmed,deaths,recovered,active,mortality
10375,,Mauritius,-20.200000,57.500000,2020-03-01,0,0,0,0,NaN
5172,,Serbia,44.016500,21.005900,2020-02-10,0,0,0,0,NaN
17886,Ningxia,China,37.269200,106.165500,2020-03-30,75,0,75,0,0.000000
21046,,Cote d'Ivoire,7.540000,-5.547100,2020-04-11,533,4,58,471,0.007505
5277,Grand Princess,Canada,37.648900,-122.665500,2020-02-11,0,0,0,0,NaN
18647,Saskatchewan,Canada,52.939900,-106.450900,2020-04-02,206,3,0,203,0.014563
21418,,Singapore,1.283300,103.833300,2020-04-12,2532,8,560,1964,0.003160
5569,Jilin,China,43.666100,126.192300,2020-02-12,83,1,22,60,0.012048
3911,,Saint Kitts and Nevis,17.357822,-62.782998,2020-02-05,0,0,0,0,NaN
3823,,Malta,35.937500,14.375400,2020-02-05,0,0,0,0,NaN


In [19]:
data

,state,country,lat,long,date,confirmed,deaths,recovered,active,mortality
0,,Afghanistan,33.000000,65.000000,2020-01-22,0,0,0,0,NaN
1,,Albania,41.153300,20.168300,2020-01-22,0,0,0,0,NaN
2,,Algeria,28.033900,1.659600,2020-01-22,0,0,0,0,NaN
3,,Andorra,42.506300,1.521800,2020-01-22,0,0,0,0,NaN
4,,Angola,-11.202700,17.873900,2020-01-22,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...
24099,Saint Pierre and Miquelon,France,46.885200,-56.315900,2020-04-22,1,0,0,1,0.0
24100,,South Sudan,6.877000,31.307000,2020-04-22,4,0,0,4,0.0
24101,,Western Sahara,24.215500,-12.885800,2020-04-22,6,0,0,6,0.0
24102,,Sao Tome and Principe,0.186360,6.613081,2020-04-22,4,0,0,4,0.0


#### Reading Population Data
****In this Data the Population is 2020 census****

##### Cleaning the Data

In [20]:
def p2f(x):
    """
    Convert urban percentage to float
    """
    try:
        return float(x.strip('%'))/100
    except:
        return np.nan

def age2int(x):
    """
    Convert Age to integer
    """
    try:
        return int(x)
    except:
        return np.nan

def fert2float(x):
    """
    Convert Fertility Rate to float
    """
    try:
        return float(x)
    except:
        return np.nan

## Reding the Population Data and using converters

data_pop = pd.read_csv("population_by_country_2020.csv", 
                           converters={'Urban Pop %':p2f,
                           'Fert. Rate':fert2float,
                           'Med. Age':age2int})

In [21]:
data_pop

,Country (or dependency),Population (2020),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share
0,China,1439323776,0.39%,5540090,153,9388211,-348399.0,1.7,38.0,0.61,18.47%
1,India,1380004385,0.99%,13586631,464,2973190,-532687.0,2.2,28.0,0.35,17.70%
2,United States,331002651,0.59%,1937734,36,9147420,954806.0,1.8,38.0,0.83,4.25%
3,Indonesia,273523615,1.07%,2898047,151,1811570,-98955.0,2.3,30.0,0.56,3.51%
4,Pakistan,220892340,2.00%,4327022,287,770880,-233379.0,3.6,23.0,0.35,2.83%
...,...,...,...,...,...,...,...,...,...,...,...
230,Montserrat,4992,0.06%,3,50,100,NaN,NaN,NaN,0.10,0.00%
231,Falkland Islands,3480,3.05%,103,0,12170,NaN,NaN,NaN,0.66,0.00%
232,Niue,1626,0.68%,11,6,260,NaN,NaN,NaN,0.46,0.00%
233,Tokelau,1357,1.27%,17,136,10,NaN,NaN,NaN,0.00,0.00%


In [22]:
display(data_pop.shape)
display(data_pop.info())
display(data_pop.head())
display(data_pop.describe())

(235, 11)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235 entries, 0 to 234
Data columns (total 11 columns):
Country (or dependency)    235 non-null object
Population (2020)          235 non-null int64
Yearly Change              235 non-null object
Net Change                 235 non-null int64
Density (P/Km²)            235 non-null int64
Land Area (Km²)            235 non-null int64
Migrants (net)             201 non-null float64
Fert. Rate                 201 non-null float64
Med. Age                   201 non-null float64
Urban Pop %                222 non-null float64
World Share                235 non-null object
dtypes: float64(4), int64(4), object(3)
memory usage: 20.3+ KB


None

,Country (or dependency),Population (2020),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share
0,China,1439323776,0.39%,5540090,153,9388211,-348399.0,1.7,38.0,0.61,18.47%
1,India,1380004385,0.99%,13586631,464,2973190,-532687.0,2.2,28.0,0.35,17.70%
2,United States,331002651,0.59%,1937734,36,9147420,954806.0,1.8,38.0,0.83,4.25%
3,Indonesia,273523615,1.07%,2898047,151,1811570,-98955.0,2.3,30.0,0.56,3.51%
4,Pakistan,220892340,2.00%,4327022,287,770880,-233379.0,3.6,23.0,0.35,2.83%


,Population (2020),Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %
count,2.350000e+02,2.350000e+02,235.000000,2.350000e+02,201.000000,201.000000,201.000000,222.000000
mean,3.316936e+07,3.460878e+05,475.770213,5.535918e+05,6.283582,2.693035,30.606965,0.592342
std,1.351374e+08,1.128260e+06,2331.285935,1.687796e+06,123291.887548,1.264496,9.127965,0.242304
min,8.010000e+02,-3.838400e+05,0.000000,0.000000e+00,-653249.000000,1.100000,15.000000,0.000000
25%,3.988760e+05,4.240000e+02,37.000000,2.545000e+03,-10047.000000,1.700000,22.000000,0.430000
50%,5.459642e+06,3.917000e+04,95.000000,7.724000e+04,-852.000000,2.300000,30.000000,0.605000
75%,2.057705e+07,2.496600e+05,239.500000,4.038200e+05,9741.000000,3.600000,39.000000,0.790000
max,1.439324e+09,1.358663e+07,26337.000000,1.637687e+07,954806.000000,7.000000,48.000000,1.000000


In [23]:
data_pop.rename(columns={'Country (or dependency)': 'country',
                             'Population (2020)' : 'population',
                             'Density (P/Km²)' : 'density',
                             'Fert. Rate' : 'fertility',
                             'Med. Age' : "age",
                             'Urban Pop %' : 'urban percentage'}, inplace=True)
data_pop['country'] = data_pop['country'].replace('United States', 'US')
data_pop = data_pop[["country", "population", "density", "fertility", "age", "urban percentage"]]

In [24]:
display(data_pop.shape)
display(data_pop.info())
display(data_pop.head())
display(data_pop.describe())

(235, 6)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235 entries, 0 to 234
Data columns (total 6 columns):
country             235 non-null object
population          235 non-null int64
density             235 non-null int64
fertility           201 non-null float64
age                 201 non-null float64
urban percentage    222 non-null float64
dtypes: float64(3), int64(2), object(1)
memory usage: 11.1+ KB


None

,country,population,density,fertility,age,urban percentage
0,China,1439323776,153,1.7,38.0,0.61
1,India,1380004385,464,2.2,28.0,0.35
2,US,331002651,36,1.8,38.0,0.83
3,Indonesia,273523615,151,2.3,30.0,0.56
4,Pakistan,220892340,287,3.6,23.0,0.35


,population,density,fertility,age,urban percentage
count,2.350000e+02,235.000000,201.000000,201.000000,222.000000
mean,3.316936e+07,475.770213,2.693035,30.606965,0.592342
std,1.351374e+08,2331.285935,1.264496,9.127965,0.242304
min,8.010000e+02,0.000000,1.100000,15.000000,0.000000
25%,3.988760e+05,37.000000,1.700000,22.000000,0.430000
50%,5.459642e+06,95.000000,2.300000,30.000000,0.605000
75%,2.057705e+07,239.500000,3.600000,39.000000,0.790000
max,1.439324e+09,26337.000000,7.000000,48.000000,1.000000


#### Merging the Population Data with the original data

In [25]:
data = pd.merge(data, data_pop, on='country')

In [26]:
data

,state,country,lat,long,date,confirmed,deaths,recovered,active,mortality,population,density,fertility,age,urban percentage
0,,Afghanistan,33.000000,65.000000,2020-01-22,0,0,0,0,NaN,38928346,60,4.6,18.0,0.25
1,,Afghanistan,33.000000,65.000000,2020-01-23,0,0,0,0,NaN,38928346,60,4.6,18.0,0.25
2,,Afghanistan,33.000000,65.000000,2020-01-24,0,0,0,0,NaN,38928346,60,4.6,18.0,0.25
3,,Afghanistan,33.000000,65.000000,2020-01-25,0,0,0,0,NaN,38928346,60,4.6,18.0,0.25
4,,Afghanistan,33.000000,65.000000,2020-01-26,0,0,0,0,NaN,38928346,60,4.6,18.0,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22811,,Yemen,15.552727,48.516388,2020-04-18,1,0,0,1,0.0,29825964,56,3.8,20.0,0.38
22812,,Yemen,15.552727,48.516388,2020-04-19,1,0,0,1,0.0,29825964,56,3.8,20.0,0.38
22813,,Yemen,15.552727,48.516388,2020-04-20,1,0,0,1,0.0,29825964,56,3.8,20.0,0.38
22814,,Yemen,15.552727,48.516388,2020-04-21,1,0,0,1,0.0,29825964,56,3.8,20.0,0.38


In [27]:
data['infectionRate'] = round((data['confirmed']/data['population'])*100, 5)

In [28]:
data

,state,country,lat,long,date,confirmed,deaths,recovered,active,mortality,population,density,fertility,age,urban percentage,infectionRate
0,,Afghanistan,33.000000,65.000000,2020-01-22,0,0,0,0,NaN,38928346,60,4.6,18.0,0.25,0.0
1,,Afghanistan,33.000000,65.000000,2020-01-23,0,0,0,0,NaN,38928346,60,4.6,18.0,0.25,0.0
2,,Afghanistan,33.000000,65.000000,2020-01-24,0,0,0,0,NaN,38928346,60,4.6,18.0,0.25,0.0
3,,Afghanistan,33.000000,65.000000,2020-01-25,0,0,0,0,NaN,38928346,60,4.6,18.0,0.25,0.0
4,,Afghanistan,33.000000,65.000000,2020-01-26,0,0,0,0,NaN,38928346,60,4.6,18.0,0.25,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22811,,Yemen,15.552727,48.516388,2020-04-18,1,0,0,1,0.0,29825964,56,3.8,20.0,0.38,0.0
22812,,Yemen,15.552727,48.516388,2020-04-19,1,0,0,1,0.0,29825964,56,3.8,20.0,0.38,0.0
22813,,Yemen,15.552727,48.516388,2020-04-20,1,0,0,1,0.0,29825964,56,3.8,20.0,0.38,0.0
22814,,Yemen,15.552727,48.516388,2020-04-21,1,0,0,1,0.0,29825964,56,3.8,20.0,0.38,0.0


### ICU Beds Per Country
We wish to further inspect the ratio of ICU beds per 1000 people that every country has readily available. Therefore we load the dataset from: https://www.kaggle.com/hamzael1/hospital-beds-by-country

#### Reading ICU Beds Data
##### Data Set Name - data_icu_clean
##### Columns - 

In [29]:
data_icu = pd.read_csv("ICU beds.csv")

In [30]:
data_icu['Country Name'] = data_icu['Country Name'].replace('United States', 'US')
data_icu['Country Name'] = data_icu['Country Name'].replace('Russian Federation', 'Russia')
data_icu['Country Name'] = data_icu['Country Name'].replace('Iran, Islamic Rep.', 'Iran')
data_icu['Country Name'] = data_icu['Country Name'].replace('Egypt, Arab Rep.', 'Egypt')
data_icu['Country Name'] = data_icu['Country Name'].replace('Venezuela, RB', 'Venezuela')
data['country'] = data['country'].replace('Czechia', 'Czech Republic')

In [31]:
# We wish to have the most recent values, thus we need to go through every year and extract the most recent one, if it exists.
data_icu_clean = pd.DataFrame()
data_icu_clean["country"] = data_icu["Country Name"]
data_icu_clean["icu"] = np.nan

for year in range(1960, 2020):
    year_df = data_icu[str(year)].dropna()
    data_icu_clean["icu"].loc[year_df.index] = year_df.values

In [32]:
data_icu_clean.sample(2)

,country,icu
152,Mexico,1.5
79,United Kingdom,2.8


#### Merging the Population Data with the original data

In [33]:
display(data.info())
display(data_icu_clean.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22816 entries, 0 to 22815
Data columns (total 16 columns):
state               22816 non-null object
country             22816 non-null object
lat                 22816 non-null float64
long                22816 non-null float64
date                22816 non-null datetime64[ns]
confirmed           22816 non-null int64
deaths              22816 non-null int64
recovered           22816 non-null int64
active              22816 non-null int64
mortality           13557 non-null float64
population          22816 non-null int64
density             22816 non-null int64
fertility           22264 non-null float64
age                 22264 non-null float64
urban percentage    22356 non-null float64
infectionRate       22816 non-null float64
dtypes: datetime64[ns](1), float64(7), int64(6), object(2)
memory usage: 3.0+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264 entries, 0 to 263
Data columns (total 2 columns):
country    264 non-null object
icu        246 non-null float64
dtypes: float64(1), object(1)
memory usage: 4.2+ KB


None

In [34]:
data = pd.merge(data, data_icu_clean, on='country')

In [35]:
display(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21712 entries, 0 to 21711
Data columns (total 17 columns):
state               21712 non-null object
country             21712 non-null object
lat                 21712 non-null float64
long                21712 non-null float64
date                21712 non-null datetime64[ns]
confirmed           21712 non-null int64
deaths              21712 non-null int64
recovered           21712 non-null int64
active              21712 non-null int64
mortality           13080 non-null float64
population          21712 non-null int64
density             21712 non-null int64
fertility           21252 non-null float64
age                 21252 non-null float64
urban percentage    21344 non-null float64
infectionRate       21712 non-null float64
icu                 21528 non-null float64
dtypes: datetime64[ns](1), float64(8), int64(6), object(2)
memory usage: 3.0+ MB


None

In [36]:
display(data.head())
display(data.describe())

,state,country,lat,long,date,confirmed,deaths,recovered,active,mortality,population,density,fertility,age,urban percentage,infectionRate,icu
0,,Afghanistan,33.0,65.0,2020-01-22,0,0,0,0,NaN,38928346,60,4.6,18.0,0.25,0.0,0.5
1,,Afghanistan,33.0,65.0,2020-01-23,0,0,0,0,NaN,38928346,60,4.6,18.0,0.25,0.0,0.5
2,,Afghanistan,33.0,65.0,2020-01-24,0,0,0,0,NaN,38928346,60,4.6,18.0,0.25,0.0,0.5
3,,Afghanistan,33.0,65.0,2020-01-25,0,0,0,0,NaN,38928346,60,4.6,18.0,0.25,0.0,0.5
4,,Afghanistan,33.0,65.0,2020-01-26,0,0,0,0,NaN,38928346,60,4.6,18.0,0.25,0.0,0.5


,lat,long,confirmed,deaths,recovered,active,mortality,population,density,fertility,age,urban percentage,infectionRate,icu
count,21712.000000,21712.000000,21712.000000,21712.000000,21712.000000,21712.000000,13080.000000,2.171200e+04,21712.000000,21252.000000,21252.000000,21344.000000,21712.000000,21528.000000
mean,21.595297,22.620812,2256.096721,134.098195,548.566737,1573.434276,0.021856,2.350853e+08,303.182203,2.361039,33.610390,0.648448,0.010136,3.281300
std,25.477109,71.993711,21234.734064,1378.410600,4759.669756,17283.085021,0.053682,4.952269e+08,1796.345003,1.140941,8.554822,0.201390,0.058337,2.160666
min,-51.796300,-135.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,3.393100e+04,2.000000,1.200000,15.000000,0.130000,0.000000,0.100000
25%,7.709875,-44.934550,0.000000,0.000000,0.000000,0.000000,0.000000,5.729332e+06,30.750000,1.700000,28.000000,0.550000,0.000000,1.600000
50%,23.914900,21.375600,5.000000,0.000000,0.000000,3.000000,0.004673,2.549988e+07,109.500000,1.800000,38.000000,0.645000,0.000010,2.800000
75%,41.435275,84.497525,175.000000,2.000000,16.000000,85.000000,0.023622,6.788601e+07,153.000000,2.500000,40.000000,0.820000,0.000420,4.200000
max,71.706900,178.065000,839675.000000,46583.000000,99400.000000,715726.000000,1.000000,1.439324e+09,26337.000000,7.000000,48.000000,0.980000,1.438210,13.800000


### Temperature Data
In our next step, we wish to analyze the weather and temperature data of the respective countries since the outbreak of the virus. We have composed a dataset here: https://www.kaggle.com/winterpierre91/covid19-global-weather-data

I believe there would some colleration between certain weather metrics and the speed of the number of infections/deaths.

In [37]:
data_temp = pd.read_csv("temperature Dataframe.csv")

In [38]:
data_temp.columns

Index(['Unnamed: 0', 'id', 'province', 'country', 'lat', 'long', 'date',
       'cases', 'fatalities', 'capital', 'humidity', 'sunHour', 'tempC',
       'windspeedKmph'],
      dtype='object')

#### Cleaning Weather Data

In [39]:
data_temp['country'] = data_temp['country'].replace('USA', 'US')
data_temp['country'] = data_temp['country'].replace('UK', 'United Kingdom')
data_temp = data_temp[["country", "province", "date", "humidity", "sunHour", "tempC", "windspeedKmph"]].reset_index()
data_temp.rename(columns={'province': 'state'}, inplace=True)
data_temp["date"] = pd.to_datetime(data_temp['date'])
data_temp['state'] = data_temp['state'].fillna('')

In [40]:
display(data_temp.shape)
display(data_temp.info())
display(data_temp.head())
display(data_temp.describe())

(16677, 8)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16677 entries, 0 to 16676
Data columns (total 8 columns):
index            16677 non-null int64
country          16677 non-null object
state            16677 non-null object
date             16677 non-null datetime64[ns]
humidity         16500 non-null float64
sunHour          16500 non-null float64
tempC            16500 non-null float64
windspeedKmph    16500 non-null float64
dtypes: datetime64[ns](1), float64(4), int64(1), object(2)
memory usage: 1.0+ MB


None

,index,country,state,date,humidity,sunHour,tempC,windspeedKmph
0,0,Afghanistan,,2020-01-22,65.0,8.7,-1.0,8.0
1,1,Afghanistan,,2020-01-23,59.0,8.7,-3.0,8.0
2,2,Afghanistan,,2020-01-24,71.0,7.1,0.0,7.0
3,3,Afghanistan,,2020-01-25,79.0,8.7,0.0,7.0
4,4,Afghanistan,,2020-01-26,64.0,8.7,-1.0,8.0


,index,humidity,sunHour,tempC,windspeedKmph
count,16677.000000,16500.000000,16500.000000,16500.000000,16500.000000
mean,8338.000000,66.494182,8.529776,16.027879,12.625879
std,4814.379555,18.980878,2.523400,10.582865,7.657534
min,0.000000,5.000000,1.500000,-21.000000,1.000000
25%,4169.000000,56.000000,6.200000,9.000000,7.000000
50%,8338.000000,72.000000,8.700000,14.000000,11.000000
75%,12507.000000,80.000000,11.000000,25.000000,16.000000
max,16676.000000,99.000000,14.000000,45.000000,71.000000


In [41]:
data = data.merge(data_temp, on=['country','date', 'state'], how='inner')

In [42]:
display(data.shape)
display(data.info())
display(data.head())
display(data.describe())

(7852, 22)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7852 entries, 0 to 7851
Data columns (total 22 columns):
state               7852 non-null object
country             7852 non-null object
lat                 7852 non-null float64
long                7852 non-null float64
date                7852 non-null datetime64[ns]
confirmed           7852 non-null int64
deaths              7852 non-null int64
recovered           7852 non-null int64
active              7852 non-null int64
mortality           3807 non-null float64
population          7852 non-null int64
density             7852 non-null int64
fertility           7726 non-null float64
age                 7726 non-null float64
urban percentage    7684 non-null float64
infectionRate       7852 non-null float64
icu                 7810 non-null float64
index               7852 non-null int64
humidity            7770 non-null float64
sunHour             7770 non-null float64
tempC               7770 non-null float64
windspeedKmph       

None

,state,country,lat,long,date,confirmed,deaths,recovered,active,mortality,...,fertility,age,urban percentage,infectionRate,icu,index,humidity,sunHour,tempC,windspeedKmph
0,,Afghanistan,33.0,65.0,2020-01-22,0,0,0,0,NaN,...,4.6,18.0,0.25,0.0,0.5,0,65.0,8.7,-1.0,8.0
1,,Afghanistan,33.0,65.0,2020-01-23,0,0,0,0,NaN,...,4.6,18.0,0.25,0.0,0.5,1,59.0,8.7,-3.0,8.0
2,,Afghanistan,33.0,65.0,2020-01-24,0,0,0,0,NaN,...,4.6,18.0,0.25,0.0,0.5,2,71.0,7.1,0.0,7.0
3,,Afghanistan,33.0,65.0,2020-01-25,0,0,0,0,NaN,...,4.6,18.0,0.25,0.0,0.5,3,79.0,8.7,0.0,7.0
4,,Afghanistan,33.0,65.0,2020-01-26,0,0,0,0,NaN,...,4.6,18.0,0.25,0.0,0.5,4,64.0,8.7,-1.0,8.0


,lat,long,confirmed,deaths,recovered,active,mortality,population,density,fertility,age,urban percentage,infectionRate,icu,index,humidity,sunHour,tempC,windspeedKmph
count,7852.000000,7852.000000,7852.000000,7852.000000,7852.000000,7852.000000,3807.000000,7.852000e+03,7852.000000,7726.000000,7726.000000,7684.000000,7852.000000,7810.000000,7852.000000,7770.000000,7770.000000,7770.000000,7770.000000
mean,22.877934,31.000446,562.982298,24.565716,188.555655,349.860927,0.016595,2.886241e+08,341.328324,2.214717,34.511520,0.656605,0.001762,3.466428,6567.584692,62.852767,8.627542,16.259846,12.200644
std,25.113476,70.650673,5274.349493,350.226300,2453.309743,3331.853933,0.070787,5.432920e+08,2015.158894,0.983484,7.901354,0.185540,0.016421,2.204365,4215.606177,21.011908,2.607200,11.486192,7.360215
min,-41.454500,-123.120700,0.000000,0.000000,0.000000,0.000000,0.000000,3.812800e+04,2.000000,1.200000,17.000000,0.150000,0.000000,0.300000,0.000000,6.000000,1.500000,-21.000000,1.000000
25%,8.538000,-7.092600,0.000000,0.000000,0.000000,0.000000,0.000000,5.792202e+06,26.000000,1.700000,29.000000,0.560000,0.000000,1.900000,2976.750000,48.000000,6.900000,7.000000,7.000000
50%,27.614000,28.283300,0.000000,0.000000,0.000000,0.000000,0.000000,2.654586e+07,108.000000,1.800000,38.000000,0.630000,0.000000,3.400000,6059.000000,68.000000,8.700000,15.000000,10.000000
75%,42.500000,101.487000,42.000000,0.000000,1.000000,18.000000,0.010988,8.433907e+07,153.000000,2.400000,41.000000,0.820000,0.000030,4.200000,9672.500000,79.000000,11.600000,26.000000,16.000000
max,64.963100,174.886000,119827.000000,14681.000000,63612.000000,85388.000000,1.000000,1.439324e+09,26337.000000,6.100000,48.000000,0.980000,0.568170,13.800000,16676.000000,99.000000,14.000000,45.000000,65.000000


## Saving the final File 
### Final File Name - countries_icu_temp

In [44]:
data.to_csv("countries_icu_temp.csv")

## Model Building